In [ ]:
!pip install snowflake-snowpark-python[pandas]

In [ ]:
!pip install snowflake-connector-python

In [ ]:
!pip install snowflake-ml-python

     |████████████████████████████████| 1.9MB 1.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl
     |████████████████████████████████| 61kB 9.7MB/s  eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/95/c7/608cbdbaa37e6f0dfb22d19e1fa9365c113aacc10cfd6748a148f91353d0/snowflake_connector_python-3.10.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     |████████████████████████████████| 81kB 13.2MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 72kB/s  eta 0:00:01     |██▉                             | 17.9MB 29.4MB/s eta 0:00:07     |███▏                            | 19.7MB 29.4MB/s eta 0:00:07     |█████████                       | 56.8MB 5.4MB/s eta 0:00:27     |██████████                      | 62.7MB 6.0MB/s eta 0:00:23     |██████████████▍                 | 90.2MB 5.7MB/s eta 0:00:20     |████████████████    

  Using cached https://files.pythonhosted.org/packages/73/6d/b5406752c4e4ba86692b22fab0afed8b48f16bdde8f92e1d852976b61dc6/tomlkit-0.12.5-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/32/46/9cb0e58b2deb7f82b84065f37f3bffeb12413f947f9388e4cac22c4621ce/sortedcontainers-2.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e5/3e/741d8c82801c347547f8a2a06aa57dbb1992be9e948df2ea0eda2c8b79e8/idna-3.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/54/a7/2104f674a5a6845b04c8ff01659becc6b8978ca410b82b94287e0b1e018b/pyOpenSSL-24.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ea/ac/e9e77bc385729035143e54cc8c4785bd480eaca9df17565963556b0b7a93/cffi-1.16.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://fil

     |████████████████████████████████| 245kB 34.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 7.4MB/s  eta 0:00:01
     |████████████████████████████████| 133kB 36.9MB/s eta 0:00:01
     |████████████████████████████████| 307kB 54.9MB/s eta 0:00:01
     |████████████████████████████████| 12.2MB 61.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 12.4MB/s eta 0:00:01


In [ ]:
from snowflake.snowpark import Session

In [ ]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [ ]:
session = get_session()
# session.close()
session

In [ ]:
REGISTRY_DATABASE_NAME = "test_database_prak"
REGISTRY_SCHEMA_NAME = "test_schema_prak"

In [1]:
from snowflake.ml.registry import model_registry

ModuleNotFoundError: No module named 'snowflake.ml'

In [ ]:
!pip freeze | grep -i snow